In [1]:
import numpy as np
from pathlib import Path
import pandas as pd
import time
import tensorflow as tf
from tensorflow.python import debug as tf_debug
from utils import download, extract, get_cifar10_batches, get_cifar10_test_batch
tf.set_random_seed(42)

In [2]:
cifar_10_download = Path("data/cifar-10-python.tar.gz")
if not cifar_10_download.exists():
    download("https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz", cifar_10_download)

In [3]:
def if_not_exists_make(path):
    p = Path(path)
    if not p.exists():
        p.mkdir()
    return p
extract_loc = Path("data/cifar-10-batches-py")
if not extract_loc.exists():
    extract(cifar_10_download)
log_dir = if_not_exists_make("logs")
train_logs = if_not_exists_make("logs/train_" + str(int(time.time())))
test_logs = if_not_exists_make("logs/test_" + str(int(time.time())))

In [4]:
batches = get_cifar10_batches()
test_batch = get_cifar10_test_batch()

In [5]:
n_epochs = 10
batch_size = 1
n = 32
n_channels = 3
n_samples = batches.shape[0]

In [6]:
inp = tf.placeholder(tf.float32, [batch_size, n, n, n_channels], name="x_input")
category = tf.placeholder(tf.int32, [batch_size, 1], name="y_input")

conv1 = tf.layers.conv2d(
    inputs=inp,
    filters=32,
    kernel_size=[5, 5],
    padding="same",
    activation=tf.nn.relu,
    name="conv_1")
pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2, name="max_pool_1")

# Convolutional Layer #2 and Pooling Layer #2
conv2 = tf.layers.conv2d(
    inputs=pool1,
    filters=64,
    kernel_size=[5, 5],
    padding="same",
    activation=tf.nn.relu,
    name="conv_2")
pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2, name="max_pool_2")

# Dense Layer
pool2_flat = tf.reshape(pool2, [-1, 8 * 8 * 64], name="flatten")
dense = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu, name="dense_1")
dropout = tf.layers.dropout(inputs=dense, rate=0.4, name="dropout")

# Logits Layer
logits = tf.layers.dense(inputs=dropout, units=10, name="logits")

global_step = tf.Variable(0, trainable=False, name="step")

learning_rate = tf.train.exponential_decay(0.1, global_step, 10, 0.95, name="learning_rate")

loss = tf.losses.sparse_softmax_cross_entropy(labels=category, logits=logits)
optimizer = tf.train.GradientDescentOptimizer(learning_rate, name="optimizer")
train_op = optimizer.minimize(loss=loss, global_step=global_step, name="training_op")

In [7]:
train_writer = tf.summary.FileWriter(str(train_logs), graph=tf.get_default_graph())

tf.summary.scalar("learning_rate", learning_rate)
tf.summary.scalar("loss", loss)
tf.summary.histogram("dense", dense)
tf.summary.histogram("conv2", conv2)
merged = tf.summary.merge_all()
run_options = tf.RunOptions(trace_level=tf.RunOptions.FULL_TRACE)
run_metadata = tf.RunMetadata()

In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess = tf_debug.TensorBoardDebugWrapperSession(sess, "localhost:9090")
    for ite in range(n_samples // batch_size):
        batch = batches.sample(n=batch_size)
        x, y = batch[1], batch[0]
        l, _, summary = sess.run([loss, train_op, merged], feed_dict={inp: np.stack(list(x)).reshape([batch_size, n, n, n_channels]), category: np.stack(list(y)).reshape([batch_size, 1])})
        train_writer.add_summary(summary, global_step=ite)
        this_test_batch = test_batch.sample(n=batch_size)
        test_x, test_y = this_test_batch[1], this_test_batch[0]
        t_l = sess.run([loss],
                       feed_dict={inp: np.stack(list(test_x)).reshape([batch_size, n, n, n_channels]), category: np.stack(list(test_y)).reshape([batch_size, 1])},
                      options=run_options,
                      run_metadata=run_metadata)
        train_writer.add_run_metadata(run_metadata, 'step%d' % ite)
        if ite % 20 == 0 and ite is not 0:
            print("iter: {}, train loss: {:.2f}, test loss: {}".format(ite, l, t_l))
        if ite == 200:
            break

4 ops no flops stats due to incomplete shapes.
